<h1><center>Air Quality and 311 Requests 2021, Part B</center></h1>

In [1]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib.ticker import FuncFormatter
import pandas as pd
import numpy as np
import seaborn as sns
from pandas.plotting import scatter_matrix
%matplotlib inline
from branca.element import Figure
from pylab import rcParams

import folium
from folium.features import DivIcon
from folium import FeatureGroup
import geopandas as gpd
import requests
import os
import glob

import time
import datetime

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

<h2>Reading Sensors Location Data</h2>

In [2]:
sensors_loc_df = pd.read_csv("pa_sensors.csv")
sensors_loc_df

,sensor_name,sensor_index,location_type,latitude,longitude
0,AEnv,85645,0,34.260450,-118.337010
1,AQMD_NASA_3,6454,0,34.129322,-118.202470
2,AQMD_NASA_248,27109,0,34.164770,-118.591150
3,CCA La Kretz Innovation Center LACI,29201,0,34.041237,-118.235590
4,CSUN Live Oak,29697,0,34.238180,-118.528336
5,CCA 23rd and Meyler,30391,0,33.723373,-118.296840
6,Granada Hills,87181,0,34.304108,-118.514130
7,Hacienda,7290,0,34.180466,-118.417690
8,Miracle Mile,77657,0,34.058422,-118.357635
9,Nichols Canyon Road,32969,0,34.122124,-118.360220


<h2>Getting Data Through PurpleAir API</h2>

In [3]:
READ_KEY = "A6E4A431-2262-11ED-B5AA-42010A800006" 
WRITE_KEY = "A6E54D99-2262-11ED-B5AA-42010A800006"
headers = {'X-API-Key': READ_KEY,}

In [4]:
# Check our READ_KEY
root_url = "https://api.purpleair.com/v1/keys"
api_url = root_url + f'?api_key={READ_KEY}'   
results = requests.get(api_url).json()
results

{'api_version': 'V1.0.11-0.0.39',
 'time_stamp': 1661986434,
 'api_key_type': 'READ'}

In [5]:
# See a sample of API request
s_id = 29201 # sensor index
response = requests.get(f'https://api.purpleair.com/v1/sensors/{s_id}?fields=pm2.5_cf_1%2C%20pm10.0_cf_1', headers=headers).json()
response

{'api_version': 'V1.0.11-0.0.39',
 'time_stamp': 1661986435,
 'data_time_stamp': 1661986419,
 'sensor': {'sensor_index': 29201, 'pm2.5_cf_1': 10.5, 'pm10.0_cf_1': 13.4}}

In [6]:
# See another sample of API request
root_url = "https://api.purpleair.com/v1/sensors/{}".format(s_id)
api_url = root_url + '?api_key={}'.format(READ_KEY)
results = requests.get(api_url).json()
results

{'api_version': 'V1.0.11-0.0.39',
 'time_stamp': 1661986435,
 'data_time_stamp': 1661986422,
 'sensor': {'sensor_index': 29201,
  'last_modified': 1611106365,
  'date_created': 1553019280,
  'last_seen': 1661986379,
  'private': 0,
  'is_owner': 0,
  'name': 'CCA\xa0La Kretz Innovation Center LACI',
  'icon': 0,
  'location_type': 0,
  'model': 'PA-II',
  'hardware': '2.0+BME280+PMSX003-B+PMSX003-A',
  'led_brightness': 35,
  'firmware_version': '7.00',
  'rssi': -73,
  'uptime': 17316,
  'pa_latency': 286,
  'memory': 15120,
  'position_rating': 5,
  'latitude': 34.041237,
  'longitude': -118.23559,
  'altitude': 259,
  'channel_state': 3,
  'channel_flags': 0,
  'channel_flags_manual': 0,
  'channel_flags_auto': 0,
  'confidence': 100,
  'confidence_auto': 100,
  'confidence_manual': 100,
  'humidity': 26,
  'humidity_a': 26,
  'temperature': 97,
  'temperature_a': 97,
  'pressure': 1000.18,
  'pressure_a': 1000.18,
  'analog_input': 0.02,
  'pm1.0': 7.5,
  'pm1.0_a': 7.9,
  'pm1.0_b

PurpleAir requires us to use __Unix Timestamp__ when making an API call. We want to get the data of `pm2.5_cf_1` and `pm10.0_cf_1` from the selected sensors from 01/01/2021 until 12/31/2022. See information on how to convert DateTime to UNIX Timestamp and how to convert the UNIX Timestamp back to Datetime [here](https://www.geeksforgeeks.org/how-to-convert-datetime-to-unix-timestamp-in-python/).

<h4>Get UNIX Timestamp of Start Date</h4>

In [7]:
# assigned regular string date (year, month, date, hour, minute)
start_date_time = datetime.datetime(2021, 1, 1, 16, 0) 
 
# print regular python date&time
print("start_date_time =>", start_date_time)
 
# displaying unix timestamp after conversion
unix_start_time = time.mktime(start_date_time.timetuple())
print("unix_start_time => ",
      (unix_start_time))

start_date_time => 2021-01-01 16:00:00
unix_start_time =>  1609545600.0


In [8]:
# Other method to get Unix Tinestamp
date_example = "1/2/2021, 00:00:00"
date_format = datetime.datetime.strptime(date_example,
                                         "%m/%d/%Y, %H:%M:%S")
unix_time = datetime.datetime.timestamp(date_format)
print(unix_time)

1609574400.0


<h4>Get UNIX Timestamp of End Date</h4>

In [9]:
# assigned regular string date (year, month, date, hour, minute, second)
end_date_time = datetime.datetime(2022, 1, 1, 16, 59, 59) 
 
# print regular python date&time
print("end_date_time =>",end_date_time)
 
# displaying unix timestamp after conversion
unix_end_time = time.mktime(end_date_time.timetuple())
print("unix_end_time => ",
      (unix_end_time))

end_date_time => 2022-01-01 16:59:59
unix_end_time =>  1641085199.0


In [10]:
# Check our UNIX Timestamp by converting it back to DateTime
date_time = datetime.datetime.fromtimestamp(unix_end_time)
 
# print unix time stamp
print("Unix_Time =>",unix_end_time)
print(type(unix_end_time))
 
# displaying date and time in a regular
# string format
print("Date & Time =>" ,
      date_time.strftime('%Y-%m-%d %H:%M:%S'))

Unix_Time => 1641085199.0
<class 'float'>
Date & Time => 2022-01-01 16:59:59


In [11]:
# Import function from pa_tools.py
from pa_tools import create_df

Let's see what a full year data look like from __*CCA La Kretz Innovation Center LACI*__.

PurpleAir only provides raw data of PMs. Therefore, we need to calculate the AQI from the raw data. For information on how to calculate AQI from raw data see [EPA AQI Breakpoints](https://aqs.epa.gov/aqsweb/documents/codetables/aqi_breakpoints.html) and [The AQI Equation](https://forum.airnowtech.org/t/the-aqi-equation/169).

In [12]:
s_id = 29201 # sensor_index
average = 1440 # get the average of raw data per 1440 minutes or 1 day (1 day = 1440 minutes)
response = requests.get(f"https://api.purpleair.com/v1/sensors/{s_id}/history?start_timestamp={unix_start_time}&end_timestamp={unix_end_time}&average={average}&fields=pm1.0_cf_1%2C%20pm2.5_cf_1%2C%20pm10.0_cf_1", headers=headers).json()
response    

{'api_version': 'V1.0.11-0.0.39',
 'time_stamp': 1661986436,
 'sensor_index': 29201,
 'start_timestamp': 1609545600,
 'end_timestamp': 1641085199,
 'average': 1440,
 'fields': ['time_stamp', 'pm1.0_cf_1', 'pm2.5_cf_1', 'pm10.0_cf_1'],
 'data': [[1637971200, 9.8515, 14.1795, 17.1305],
  [1632355200, 9.215, 13.4055, 16.768],
  [1637539200, 3.618, 4.9655000000000005, 6.045999999999999],
  [1620950400, 14.157499999999999, 21.866500000000002, 25.368499999999997],
  [1627344000, 14.233, 22.6375, 26.703],
  [1631232000, 11.009, 17.051000000000002, 20.011499999999998],
  [1619308800, 6.606999999999999, 10.111, 12.828],
  [1615593600, 15.974499999999999, 22.2845, 25.006999999999998],
  [1612742400, 32.635999999999996, 57.581999999999994, 70.9885],
  [1627603200, 12.318999999999999, 18.9165, 22.3215],
  [1633651200, 3.981, 5.7940000000000005, 7.068],
  [1633305600, 11.314499999999999, 15.239, 17.4745],
  [1618617600, 13.411, 20.197000000000003, 25.22],
  [1614297600, 5.375500000000001, 7.399, 9.

<h3>Getting Data of the Selected 10 Sensors</h3>

We will create dataframe from the selected 10 sensors. The dataframe will consist of `pm1.0_cf_1`, `pm2.5_cf_1`, and `pm10.0_cf_1`. We will calculate the AQI of pm2.5_cf_1 and pm10.0_cf_1, then we will find MAX(pm2.5 AQI, pm10.0 AQI) to get the final AQI.

In [13]:
# This part is commented so it won't call the API everytime we refresh the notebook.

#sensors_list = sensors_loc_df['sensor_index']
#average = 1440

#fnames = []
#for idx in sensors_list:
#    response = requests.get(f"https://api.purpleair.com/v1/sensors/{idx}/history?start_timestamp={unix_start_time}&end_timestamp={unix_end_time}&average={average}&fields=pm1.0_cf_1%2C%20pm2.5_cf_1%2C%20pm10.0_cf_1", headers=headers).json()
#    fname = create_df(response, idx, sensors_loc_df)
#    fnames.append(fname)

In [14]:
# This part is commented so it won't run everytime we refresh the notebook.

# Print the created csv files
#fnames

In [15]:
# use glob to get all the csv files in the folder
path = 'PurpleAir/Sensors_Data/pa_API/'
csv_files = glob.glob(os.path.join(path, "*.csv"))

# Create an empty list
temp_list = []

for f in csv_files:    
    path, filename = os.path.split(f)
    root, ext = os.path.splitext(filename)
    root = pd.read_csv(f)
    # Append each dataframe into df_sensors
    temp_list.append(root)
    
# Display two rowns of each dataset
for dataset in temp_list:
    display(dataset.head(2))

,date_local,pm1.0_cf_1,pm2.5_cf_1,pm10.0_cf_1,pm2.5_aqi,pm10.0_aqi,aqi,sensor_index,sensor_name
0,2021-01-01 16:00:00,3.9435,6.0165,6.8290,25,6,25,27109,AQMD_NASA_248
1,2021-01-02 16:00:00,14.0400,21.6700,24.5025,71,23,71,27109,AQMD_NASA_248


,date_local,pm1.0_cf_1,pm2.5_cf_1,pm10.0_cf_1,pm2.5_aqi,pm10.0_aqi,aqi,sensor_index,sensor_name
0,2021-01-01 16:00:00,14.9905,20.5255,23.4710,69,22,69,29201,CCA La Kretz Innovation Center LACI
1,2021-01-02 16:00:00,23.7945,35.9975,42.4185,102,39,102,29201,CCA La Kretz Innovation Center LACI


,date_local,pm1.0_cf_1,pm2.5_cf_1,pm10.0_cf_1,pm2.5_aqi,pm10.0_aqi,aqi,sensor_index,sensor_name
0,2021-01-01 16:00:00,5.2395,7.211,8.042,30,7,30,29697,CSUN Live Oak
1,2021-01-02 16:00:00,16.7065,23.393,25.950,75,24,75,29697,CSUN Live Oak


,date_local,pm1.0_cf_1,pm2.5_cf_1,pm10.0_cf_1,pm2.5_aqi,pm10.0_aqi,aqi,sensor_index,sensor_name
0,2021-01-01 16:00:00,11.208,15.0875,16.9940,57,16,57,30391,CCA 23rd and Meyler
1,2021-01-02 16:00:00,19.970,28.9735,33.3475,86,31,86,30391,CCA 23rd and Meyler


,date_local,pm1.0_cf_1,pm2.5_cf_1,pm10.0_cf_1,pm2.5_aqi,pm10.0_aqi,aqi,sensor_index,sensor_name
0,2021-01-01 16:00:00,5.2475,7.6945,8.1865,32,8,32,32969,Nichols Canyon Road
1,2021-01-02 16:00:00,16.1805,25.6050,27.3705,79,25,79,32969,Nichols Canyon Road


,date_local,pm1.0_cf_1,pm2.5_cf_1,pm10.0_cf_1,pm2.5_aqi,pm10.0_aqi,aqi,sensor_index,sensor_name
0,2021-01-01 16:00:00,6.682,9.019,9.3860,38,9,38,6454,AQMD_NASA_3
1,2021-01-02 16:00:00,25.047,36.619,38.1405,104,35,104,6454,AQMD_NASA_3


,date_local,pm1.0_cf_1,pm2.5_cf_1,pm10.0_cf_1,pm2.5_aqi,pm10.0_aqi,aqi,sensor_index,sensor_name
0,2021-01-01 16:00:00,20.3790,25.7175,27.4285,80,25,80,7290,Hacienda
1,2021-01-02 16:00:00,31.8275,42.9240,46.6265,119,43,119,7290,Hacienda


,date_local,pm1.0_cf_1,pm2.5_cf_1,pm10.0_cf_1,pm2.5_aqi,pm10.0_aqi,aqi,sensor_index,sensor_name
0,2021-01-01 16:00:00,12.3365,17.560,18.521,62,17,62,77657,Miracle Mile
1,2021-01-02 16:00:00,19.6165,30.828,32.898,90,30,90,77657,Miracle Mile


,date_local,pm1.0_cf_1,pm2.5_cf_1,pm10.0_cf_1,pm2.5_aqi,pm10.0_aqi,aqi,sensor_index,sensor_name
0,2021-01-01 16:00:00,3.0220,4.842,5.5135,20,5,20,85645,AEnv
1,2021-01-02 16:00:00,13.1195,20.633,22.9205,69,21,69,85645,AEnv


,date_local,pm1.0_cf_1,pm2.5_cf_1,pm10.0_cf_1,pm2.5_aqi,pm10.0_aqi,aqi,sensor_index,sensor_name
0,2021-01-01 16:00:00,2.5760,4.0315,4.588,17,4,17,87181,Granada Hills
1,2021-01-02 16:00:00,16.9955,26.0860,28.927,80,27,80,87181,Granada Hills


In [16]:
sensors_aqi_df = pd.concat(temp_list, axis = 0)
sensors_aqi_df.to_csv('sensors_aqi.csv', index=False)

In [17]:
# Check if there is any missing values
sensors_aqi_df.isnull().sum()

#for i in range(len(df_sensors)):
 #   print(df_sensors[i].isnull().sum())

date_local      0
pm1.0_cf_1      0
pm2.5_cf_1      0
pm10.0_cf_1     0
pm2.5_aqi       0
pm10.0_aqi      0
aqi             0
sensor_index    0
sensor_name     0
dtype: int64

<h2>Visualize 2021 311-Services Around Each Sensors on A Map</h2>

In [18]:
# Read the csv file
df2021 = pd.read_csv("../311_data_time_series/MyLA311_Service_Request_Data_2021.csv")
df2021.head(2)

C:\Miniconda\envs\IBM_DS\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,SRNumber,CreatedDate,UpdatedDate,ActionTaken,Owner,RequestType,Status,RequestSource,MobileOS,Anonymous,AssignTo,ServiceDate,ClosedDate,AddressVerified,ApproximateAddress,Address,HouseNumber,Direction,StreetName,Suffix,ZipCode,Latitude,Longitude,Location,TBMPage,TBMColumn,TBMRow,APC,CD,CDMember,NC,NCName,PolicePrecinct
0,1-1831742951,01/01/2021 12:06:16 AM,01/02/2021 02:23:21 PM,SR Created,LASAN,Metal/Household Appliances,Closed,Call,NaN,N,EV,01/02/2021 12:00:00 AM,01/02/2021 02:18:38 PM,Y,N,"15251 W CAREY RANCH LANE, 91342",15251.0,W,CAREY RANCH,LANE,91342.0,34.289389,-118.463593,"(34.2893893478, -118.46359286)",481.0,H,6.0,North Valley APC,7.0,Monica Rodriguez,5.0,Sylmar,MISSION
1,1-1831741721,01/01/2021 12:27:17 AM,01/03/2021 01:44:27 PM,SR Created,LASAN,Metal/Household Appliances,Closed,Call,NaN,N,WV,01/02/2021 12:00:00 AM,01/03/2021 01:43:16 PM,Y,N,"23022 W BLYTHE ST, 91304",23022.0,W,BLYTHE,ST,91304.0,34.214001,-118.628669,"(34.2140011149, -118.628669123)",529.0,G,3.0,South Valley APC,12.0,John Lee,11.0,West Hills,TOPANGA


In [19]:
len(df2021)

1451351

In [20]:
# Check if there is any missing values
print(df2021.isnull().sum())

SRNumber                    0
CreatedDate                 0
UpdatedDate                 0
ActionTaken                 0
Owner                       3
RequestType                 0
Status                      0
RequestSource               0
MobileOS              1055265
Anonymous                   0
AssignTo                30989
ServiceDate             75065
ClosedDate               4956
AddressVerified             0
ApproximateAddress     526984
Address                    71
HouseNumber            276043
Direction               80880
StreetName             276043
Suffix                 103943
ZipCode                   907
Latitude                  552
Longitude                 552
Location                  552
TBMPage                   555
TBMColumn                 555
TBMRow                    555
APC                       595
CD                        563
CDMember               208076
NC                      21108
NCName                  21108
PolicePrecinct            572
dtype: int

In [21]:
df2021["Status"].unique()

array(['Closed', 'Cancelled', 'Pending', 'Open', 'Referred Out',
       'Forward'], dtype=object)

In [22]:
df2021['RequestType'].unique()

array(['Metal/Household Appliances', 'Bulky Items',
       'Illegal Dumping Pickup', 'Graffiti Removal', 'Other',
       'Homeless Encampment', 'Dead Animal Removal',
       'Single Streetlight Issue', 'Multiple Streetlight Issue',
       'Report Water Waste', 'Electronic Waste', 'Feedback'], dtype=object)

In [23]:
df2021.columns

Index(['SRNumber', 'CreatedDate', 'UpdatedDate', 'ActionTaken', 'Owner',
       'RequestType', 'Status', 'RequestSource', 'MobileOS', 'Anonymous',
       'AssignTo', 'ServiceDate', 'ClosedDate', 'AddressVerified',
       'ApproximateAddress', 'Address', 'HouseNumber', 'Direction',
       'StreetName', 'Suffix', 'ZipCode', 'Latitude', 'Longitude', 'Location',
       'TBMPage', 'TBMColumn', 'TBMRow', 'APC', 'CD', 'CDMember', 'NC',
       'NCName', 'PolicePrecinct'],
      dtype='object')

In [24]:
# Create df21 which consists of few columns from df2021
df21 = df2021[['CreatedDate', 'RequestType', 'Status', 'ServiceDate', 'Latitude', 'Longitude', 'NC',
       'NCName']]

In [25]:
# Find the number of Status in df21 which is Closed
print("Number of Status equals to Closed: ", len(df21[df21['Status'] == "Closed"]))

# Find the percentage of Status in df21 which is Closed
print("Percentage of Status equals to Closed: ", len(df21[df21['Status'] == "Closed"])/len(df21)*100)

Number of Status equals to Closed:  1396440
Percentage of Status equals to Closed:  96.2165596054986


In [26]:
# Find the number of RequestType in df21 which is Feedback
print("Number of RequestType equals to Feedback: ", len(df21[df21['RequestType'] == "Feedback"]))

# Find the percentage of RequestType in df21 which is Feedback
print("Percentage of RequestType equals to Feedback: ", len(df21[df21['RequestType'] == "Feedback"])/len(df21)*100)

Number of RequestType equals to Feedback:  408
Percentage of RequestType equals to Feedback:  0.02811173864902425


In [27]:
# Select only rows which Status are Closed.
df21 = df21[df21['Status'] == "Closed"]

# Select only rows which RequestType not equal to Feedback since Feedback won't affect air qualiality.
df21 = df21[df21['RequestType'] != "Feedback"]

In [28]:
# Check if there is any missing values
print(df21.isnull().sum())

CreatedDate        0
RequestType        0
Status             0
ServiceDate    56647
Latitude         267
Longitude        267
NC             20050
NCName         20050
dtype: int64


In [29]:
# See few rows where the ServiceDate is NaN
df21[df21["ServiceDate"].isna()].head()

,CreatedDate,RequestType,Status,ServiceDate,Latitude,Longitude,NC,NCName
23,01/01/2021 02:21:06 AM,Other,Closed,NaN,34.202681,-118.431515,20.0,Van Nuys
32,01/01/2021 05:27:11 AM,Dead Animal Removal,Closed,NaN,34.174608,-118.537314,17.0,Tarzana
33,01/01/2021 05:38:37 AM,Dead Animal Removal,Closed,NaN,34.174608,-118.537314,17.0,Tarzana
43,01/01/2021 07:11:56 AM,Other,Closed,NaN,33.987520,-118.329497,80.0,Park Mesa Heights
52,01/01/2021 07:43:19 AM,Dead Animal Removal,Closed,NaN,34.209821,-118.409395,22.0,North Hollywood West


In [30]:
# Recheck the lenght of rows which ServiceDate is NaN
len(df21[df21["ServiceDate"].isna()])

56647

After examining few rows which __ServiceDate__ is equal to __NaN__. We see that the requests were created but no service were performed. We will drop the rows which the service dates are empty since our goal is to investigate the correlation between the air quality around the selected 10 sensors and the services performed because of the 311 requests. 

In [31]:
# Select only rows which ServiceDate are not NaN.
df21 = df21[df21['ServiceDate'].notnull()]

In [32]:
# Check if there is any missing values
print(df21.isnull().sum())

CreatedDate        0
RequestType        0
Status             0
ServiceDate        0
Latitude          17
Longitude         17
NC             18691
NCName         18691
dtype: int64


In [33]:
# Select only rows which Longitude or Latitude are not NaN.
df21 = df21[df21['Latitude'].notnull()]

In [34]:
# Check if there is any missing values
print(df21.isnull().sum())

CreatedDate        0
RequestType        0
Status             0
ServiceDate        0
Latitude           0
Longitude          0
NC             18674
NCName         18674
dtype: int64


In [35]:
len(df21)

1339588

In [36]:
df21['CreatedDate'] = pd.to_datetime(df21['CreatedDate'])

In [37]:
df21['ServiceDate'] = pd.to_datetime(df21['ServiceDate'])

In [38]:
df21['Diff_Dates'] = (df21["ServiceDate"] - df21["CreatedDate"])/np.timedelta64(1, 'D')

In [39]:
# Check if there are any Diff_Dates which has negative values
any(df21['Diff_Dates'] < 0)

True

In [40]:
# Find the number of Diff_Dates in df21 which is negative
print("Number of Diff_Dates < 0: ", len(df21[df21['Diff_Dates'] < 0]))

# Find the percentage of Diff_Dates in df21 which is negative
print("Percentage of Diff_Dates < 0: ", len(df21[df21['Diff_Dates'] < 0])/len(df21)*100)

Number of Diff_Dates < 0:  214651
Percentage of Diff_Dates < 0:  16.023658020227113


In [41]:
# See few rows which Diff_Dates has negative values  
df21[df21['Diff_Dates'] < 0].head(10)

,CreatedDate,RequestType,Status,ServiceDate,Latitude,Longitude,NC,NCName,Diff_Dates
117,2021-01-01 08:49:01,Multiple Streetlight Issue,Closed,2021-01-01 00:00:00,34.240892,-118.413249,7.0,Pacoima,-0.367373
1218,2021-01-01 16:49:36,Graffiti Removal,Closed,2020-12-24 09:00:00,34.303919,-118.478329,5.0,Sylmar,-8.326111
1220,2021-01-01 16:50:21,Graffiti Removal,Closed,2020-12-24 09:01:00,34.303821,-118.478874,5.0,Sylmar,-8.325938
1222,2021-01-01 16:51:07,Graffiti Removal,Closed,2020-12-26 09:02:00,34.304217,-118.434394,5.0,Sylmar,-6.325775
1226,2021-01-01 16:51:51,Graffiti Removal,Closed,2020-12-26 09:03:00,34.305553,-118.432693,5.0,Sylmar,-6.325590
1228,2021-01-01 16:52:36,Graffiti Removal,Closed,2020-12-26 09:04:00,34.305553,-118.432693,5.0,Sylmar,-6.325417
1232,2021-01-01 16:53:21,Graffiti Removal,Closed,2020-12-26 09:05:00,34.305553,-118.432693,5.0,Sylmar,-6.325243
1234,2021-01-01 16:54:06,Graffiti Removal,Closed,2020-12-26 09:06:00,34.305553,-118.432693,5.0,Sylmar,-6.325069
1236,2021-01-01 16:54:51,Graffiti Removal,Closed,2020-12-26 09:07:00,34.305553,-118.432693,5.0,Sylmar,-6.324896
1237,2021-01-01 16:55:36,Graffiti Removal,Closed,2020-12-26 09:08:00,34.300040,-118.438984,5.0,Sylmar,-6.324722


Since it doesn't make sense to have a ServiceDate which is before the CreatedDate, we will drop the rows which `Diff_Dates` < 0.

In [42]:
# Select only rows where Diff_Dates >= 0
df21 = df21[df21['Diff_Dates'] >= 0]

# Find the length of df21
len(df21)

1124937

In [43]:
# Check if there is any missing values
print(df21.isnull().sum())

CreatedDate        0
RequestType        0
Status             0
ServiceDate        0
Latitude           0
Longitude          0
NC             16795
NCName         16795
Diff_Dates         0
dtype: int64


In [44]:
df21 = df21.reset_index(drop=True)
df21.head()

,CreatedDate,RequestType,Status,ServiceDate,Latitude,Longitude,NC,NCName,Diff_Dates
0,2021-01-01 00:06:16,Metal/Household Appliances,Closed,2021-01-02,34.289389,-118.463593,5.0,Sylmar,0.995648
1,2021-01-01 00:27:17,Metal/Household Appliances,Closed,2021-01-02,34.214001,-118.628669,11.0,West Hills,0.981053
2,2021-01-01 00:40:36,Bulky Items,Closed,2021-01-02,34.200714,-118.630509,11.0,West Hills,0.971806
3,2021-01-01 00:44:16,Bulky Items,Closed,2021-01-07,34.072331,-118.311496,119.0,Greater Wilshire,5.969259
4,2021-01-01 00:44:33,Illegal Dumping Pickup,Closed,2021-01-06,34.069119,-118.237733,128.0,Historic Cultural North,4.969062


<h3>Create a Dataframe of 311-Services Performed Around the 10 Selected Sensors</h3>

Since it is too much for the computer to loop through df21 and sensors_loc_df to collect 311 requests which surround the 10 sensors. We separate the df21 into `df21[0:200000]`, `df21[20000:400000]`, `df21[40000:600000]`, `df21[60000:800000]`, `df21[80000:1000000]`, `df21[100000:len(df21)]`, then create a dataset for each part of df21. Afterwards, we combine the datasets into one dataset. 

In [45]:
# This part is commented so it won't run everytime we refresh the notebook

#start_ind = 0
#end_ind = 200000
#print(start_ind, end_ind)

#while end_ind < len(df21): 
#    fname = f'start_ind_{start_ind}'
#    create_request_sensor_df(df21.iloc[start_ind: end_ind], sensors_loc_df, fname)
#    start_ind = end_ind
#    end_ind += 200000
    #print(start_ind, end_ind)
    
#if end_ind > len(df21):
#    fname = f'start_ind_{start_ind}'
#    create_request_sensor_df(df21.iloc[start_ind: len(df21)], sensors_loc_df, fname)
    #print(start_ind, len(df21))    

In [46]:
# use glob to get all the csv files in the folder
path = 'PurpleAir/Sensors_Data/request_around_sensors/'
csv_files = glob.glob(os.path.join(path, "*.csv"))

# Create an empty list
temp_list = []

for f in csv_files:    
    path, filename = os.path.split(f)
    root, ext = os.path.splitext(filename)
    root = pd.read_csv(f)
    # Append each dataframe into temp_list
    temp_list.append(root)
          
req_around_sensors_df = pd.concat(temp_list, axis = 0)

In [47]:
req_around_sensors_df.sort_values(["Sensor#"], ascending=[True])
req_around_sensors_df = req_around_sensors_df.reset_index(drop=True)

In [48]:
req_around_sensors_df.to_csv('req_around_sensors.csv', index=False)
len(req_around_sensors_df)

255116

In [49]:
req_around_sensors_df.head()

,CreatedDate,RequestType,ServiceDate,Latitude,Longitude,SensorName,SensorIndex,Sensor#
0,2021-01-01 10:55:25,Bulky Items,2021-01-04 00:00:00,34.264866,-118.320353,AEnv,85645,0
1,2021-01-01 10:57:45,Bulky Items,2021-01-04 00:00:00,34.266566,-118.310450,AEnv,85645,0
2,2021-01-01 12:00:39,Bulky Items,2021-01-04 00:00:00,34.269679,-118.313909,AEnv,85645,0
3,2021-01-01 13:45:49,Bulky Items,2021-01-04 00:00:00,34.266567,-118.315061,AEnv,85645,0
4,2021-01-01 13:46:40,Bulky Items,2021-01-04 00:00:00,34.266567,-118.315061,AEnv,85645,0


In [50]:
# Create a base map
longitude = -118.2518
latitude = 34.0488
m4 = folium.Map(location=[latitude, longitude], tiles = 'CartoDB positron', zoom_start=11, min_zoom=8,max_zoom=15)

# Read the geojson file
#nc_geo = gpd.read_file('Neighborhood_Councils_(Certified).geojson') 

# Add several TileLayers
folium.TileLayer('openstreetmap').add_to(m4)
folium.TileLayer('stamenterrain').add_to(m4)

# Read the geojson file
nc_geo = gpd.read_file('Neighborhood_Councils_(Certified).geojson') 
nc_geo = nc_geo[["NC_ID", "NAME", "SERVICE_RE","geometry"]]

# Create choropleth map of NC
cpleth = folium.Choropleth(geo_data = nc_geo,
            name='Choropleth',
            key_on='features.properties.Name',
            #line_color='green', 
            #fill_color='YlGn',
            fill_opacity=0.1, line_opacity=0.3
            ).add_to(m4) 
folium.LayerControl().add_to(m4) 

# Set the radius of the circles surrounding the monitoring stations in meter
miles = 2
meters_c = miles * 1609.34

# Plot the selected PurpleAir sensors
for i, lat, lng, sensor_name, sensor_index in zip(sensors_loc_df.index.values.tolist(),sensors_loc_df.latitude, sensors_loc_df.longitude, sensors_loc_df.sensor_name, sensors_loc_df.sensor_index):
    tooltip_text = sensor_name+", ID: " + str(sensor_index)
    folium.CircleMarker([lat, lng], radius = 15, color = "green", weight=1,
                fill = True, fill_color = "green", fill_opacity = 0.1,
                tooltip=tooltip_text).add_to(m4)
    folium.Circle([lat, lng], radius = meters_c, color = "blue",  weight=1.5,
                             ).add_to(m4)
    folium.Marker([lat, lng],
    icon=folium.DivIcon(html=f"""<div style="font-family: Arial; 
    color: b; font-weight: bold; font size="+5"">{i}</div>""")
    ).add_to(m4)

In [51]:
req_around_sensors_df["RequestType"].unique()

array(['Bulky Items', 'Metal/Household Appliances', 'Graffiti Removal',
       'Homeless Encampment', 'Electronic Waste',
       'Illegal Dumping Pickup', 'Multiple Streetlight Issue',
       'Single Streetlight Issue', 'Dead Animal Removal'], dtype=object)

req_types = ['Bulky Items', 'Metal/Household Appliances', 'Graffiti Removal',
       'Homeless Encampment', 'Electronic Waste',
       'Illegal Dumping Pickup', 'Multiple Streetlight Issue',
       'Single Streetlight Issue', 'Dead Animal Removal']

types_color = ["blue", "darkred", "orange", "lightred", "green", "cyan", "lightgreen", "pink", "black"]

for lat, lng, req_type  in zip(req_around_sensors_df.Latitude, req_around_sensors_df.Longitude, req_around_sensors_df.RequestType):
    tooltip_text = req_type
    for i in range(len(req_types)):
        if req_type == req_types[i]:
            folium.CircleMarker([lat, lng], radius = 2, color = types_color[i], weight=1,
                fill = True, fill_color = types_color[i], fill_opacity = 0.6,
                tooltip=tooltip_text).add_to(m4)
            
m4.save('requests_around_sensors.html')
m4